CHALLENGE

Import Dependencies

In [5]:
import cv2
import uuid
import os
import time
import sys


Label Collection Images Collection

In [6]:
labels = ["Cars"]

In [7]:
Images_path = os.path.join("data","images")
if not os.path.exists(Images_path):
    os.mkdir(Images_path)

for label in labels:
    path = os.path.join(Images_path,label)
    if not os.path.exists(path):
        os.mkdir(path)


Capture Images

In [8]:
for label in labels:
    capture = cv2.VideoCapture("video_01.mp4")
    framerate = capture.get(60)
    print("Collecting images for {}".format(label))
    counter =1
    time.sleep(3)
    while capture.isOpened():
        
        sucess,frame = capture.read()
        if counter%20==0: # every 20 frames
            print("Collecting images for {}".format(counter))
            img_name = os.path.join(Images_path,label,label+"."+"{}.jpg".format(str(uuid.uuid1())))
            cv2.imwrite(img_name,frame)
            # cv2.imshow('frame', frame)
    

        if sucess == False:
            break
        counter+=1    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
capture.release()
cv2.destroyAllWindows()


Image Labelling
Labelling done using LabelIMG

In [6]:
# LABELLING_PATH = os.path.join("data","labelling")
# if not os.path.exists(LABELLING_PATH):
#     os.mkdir(LABELLING_PATH)



TESTING AND TRAINING SET SPLIT

In [7]:
TRAIN_PATH = os.path.join("data","Train")
TEST_PATH= os.path.join("data","Test")

In [12]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'


paths = {
    'WORKSPACE_PATH': os.path.join('model', 'workspace'),
    'SCRIPTS_PATH': os.path.join('model', 'scripts'),
    'APIMODEL_PATH': os.path.join('model', 'models'),
    'ANNOTATION_PATH': os.path.join('model', 'workspace', 'annotations'),
    'IMAGE_PATH': os.path.join('model', 'workspace', 'images'),
    'MODEL_PATH': os.path.join('model', 'workspace', 'models'),
    'PRETRAINED_MODEL_PATH': os.path.join('model', 'workspace', 'pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('model', 'workspace', 'models', CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('model', 'workspace', 'models', CUSTOM_MODEL_NAME, 'export'),
    'TFJS_PATH': os.path.join('model', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfjsexport'),
    'TFLITE_PATH': os.path.join('model', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH': os.path.join('model', 'protoc')
}

files = {
    'PIPELINE_CONFIG': os.path.join('model', 'workspace', 'models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}

In [9]:
#https://github.com/tensorflow/models/tree/master/research/object_detection
#  model :tf2_detection_zoo
# model : SSD MobileNet V2 FPNLite 320x320

import tensorflow as tf

print(tf.__version__)

2.7.0


In [27]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2021-11-13 23:37:58--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.167.112, 2404:6800:4006:80b::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.167.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  4.71MB/s    in 5.3s    

2021-11-13 23:38:04 (3.69 MB/s) - 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_

In [28]:
labels = [{'name':'Cars', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [29]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}
    

In [30]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

Successfully created the TFRecord file: model/workspace/annotations/train.record
Successfully created the TFRecord file: model/workspace/annotations/test.record


Copy Model Config to Training Folder

In [31]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}


In [10]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


In [46]:


config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])


In [34]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)


In [35]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(
    paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'test.record')]


In [36]:

config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)


TRAIN MODEL

In [21]:
TRAINING_SCRIPT = os.path.join(
    paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')


In [22]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(
    TRAINING_SCRIPT, paths['CHECKPOINT_PATH'], files['PIPELINE_CONFIG'])


from this directory RUN
python ~/git/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=model/workspace/models/my_ssd_mobnet --pipeline_config_path=model/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=2000


EVALUATE MODEL:


In [37]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(
    TRAINING_SCRIPT, paths['CHECKPOINT_PATH'], files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])


In [38]:
print(command)


python model/models/research/object_detection/model_main_tf2.py --model_dir=model/workspace/models/my_ssd_mobnet --pipeline_config_path=model/workspace/models/my_ssd_mobnet/pipeline.config --checkpoint_dir=model/workspace/models/my_ssd_mobnet


python ~/git/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=model/workspace/models/my_ssd_mobnet --pipeline_config_path=model/workspace/models/my_ssd_mobnet/pipeline.config --checkpoint_dir=model/workspace/models/my_ssd_mobnet

LOAD MODEL 


In [28]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util


# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections



In [29]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
category_index = label_map_util.create_category_index_from_labelmap(
    files['LABELMAP'])
IMAGE_PATH = os.path.join(
    paths['IMAGE_PATH'], 'test', 'Cars.229d0109-4450-11ec-b499-0433c2f55c1c.jpg')


In [30]:
IMAGE_PATH

'model/workspace/images/test/Cars.229d0109-4450-11ec-b499-0433c2f55c1c.jpg'

In [31]:
import cv2
import numpy as np
from matplotlib import pyplot as plt


In [34]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(
    np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes']+label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=.8,
    agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()
plt.savefig("Test.jpg")


<ipython-input-34-122abce74a30>:32: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


SEEING IT FROM THE VIDEO

In [39]:
cap = cv2.VideoCapture("video_01.mp4")
framerate = capture.get(60)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(
        np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(
        np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.8,
        agnostic_mode=False)

    # cv2.imshow('object detection',  cv2.resize(
    #     image_np_with_detections, (800, 600)))

    cv2.imshow('object detection',  
        image_np_with_detections)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


FREEZE GRAPH

In [40]:

FREEZE_SCRIPT = os.path.join(
    paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')


In [42]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(
    FREEZE_SCRIPT, files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])


print(command)


python model/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=model/workspace/models/my_ssd_mobnet/pipeline.config --trained_checkpoint_dir=model/workspace/models/my_ssd_mobnet --output_directory=model/workspace/models/my_ssd_mobnet/export


python ~/git/Tensorflow/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=model/workspace/models/my_ssd_mobnet/pipeline.config --trained_checkpoint_dir=model/workspace/models/my_ssd_mobnet --output_directory=model/workspace/models/my_ssd_mobnet/export

USE FINAL MODEL

In [45]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util


# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(
    model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


KeyError: '/home/esteban/git/Deep_Learning_varied/Computer Vision/Challenge/Computer Vision/model/workspace/models/my_ssd_mobnet/export/pipeline.config'